In [1]:
import simtk.openmm.app
import simtk.openmm
import simtk.unit as unit
import configparser
import numpy as np
import itertools
import scipy.spatial.distance as sdist
import os
import pdbfixer
import pandas
import subprocess
import nose

In [2]:
import openna

The location is /mnt/f/linux/script/openna
4.184


In [3]:
import simtk.openmm
pdb=simtk.openmm.app.PDBFile('clean.pdb')
top=pdb.topology
coord=pdb.positions
forcefield=simtk.openmm.app.ForceField(openna.xml)
s=forcefield.createSystem(top)

In [4]:
rna=openna.RNA.fromCoarsePDB('clean.pdb')
rna.periodic=False
rna.parseConfigurationFile(configuration_file=f'./openna.conf')

In [5]:
import forceTerm
all_forces = dict(Backbone=forceTerm.Backbone,
                 Angle=forceTerm.Angle,
                 Dihedral=forceTerm.Dihedral,
                 BaseStacking=forceTerm.BaseStacking)

for each_one in all_forces:
    force_now = all_forces[each_one](rna)
    s.addForce(force_now)

In [6]:
import numpy as np
temperature=300 * simtk.openmm.unit.kelvin
#platform_name='CUDA'

platform_name='Reference'

integrator = simtk.openmm.LangevinIntegrator(temperature, 1 / simtk.openmm.unit.picosecond, 2 * simtk.openmm.unit.femtoseconds)
platform = simtk.openmm.Platform.getPlatformByName(platform_name)
simulation = simtk.openmm.app.Simulation(top, s, integrator, platform)
simulation.context.setPositions(coord)
energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print(energy)

-40.06111592731405


In [7]:
energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print('TotalEnergy',round(energy,6),energy_unit.get_symbol())

TotalEnergy -40.061116 kJ/mol


In [8]:
energies = {}
for force_name, force in all_forces.items():
    force = force(rna)
    group=force.getForceGroup()
    state = simulation.context.getState(getEnergy=True, groups=2**group)
    energies[force_name] =state.getPotentialEnergy().value_in_unit(energy_unit)

for force_name in all_forces.keys():
    print(force_name, round(energies[force_name],6),energy_unit.get_symbol())

Backbone 0.0 kJ/mol
Angle 19.900579 kJ/mol
Dihedral 0.0 kJ/mol
BaseStacking -59.961694 kJ/mol


In [9]:
import sys
dcd_reporter=simtk.openmm.app.DCDReporter(f'output.dcd', 1000)
pdb_reporter=simtk.openmm.app.PDBReporter(f'output.pdb', 1000)
energy_reporter=simtk.openmm.app.StateDataReporter(sys.stdout, 1000, step=True,time=True,
                                                   potentialEnergy=True, temperature=True)
simulation.reporters.append(dcd_reporter)
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(energy_reporter)
#Run simulation
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(50000)

#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)"
1000,2.0000000000000013,14.084534822775268,300.4302766663619
2000,3.999999999999781,22.882335574471142,317.2118197549066
3000,5.999999999999561,19.729323650009547,244.7883028012694
4000,7.999999999999341,11.865611369767578,293.08555552935934
5000,10.000000000000009,-1.3710826884335603,261.68814836102473
6000,12.000000000000677,0.5034598374045771,264.1268907140374
7000,14.000000000001345,40.62909735252758,283.0304783018807
8000,16.00000000000201,40.79548709503656,270.12946816017563
9000,18.000000000000902,7.742212334747975,285.8532337810237
10000,19.999999999999794,28.581629829082814,264.45623568589224
11000,21.999999999998685,36.97981682078046,293.70209503438343
12000,23.999999999997577,15.924966877434741,339.1496128374557
13000,25.99999999999647,29.13536074448912,269.291322103523
14000,27.99999999999536,62.62409167539662,352.12267907006765
15000,29.99999999999425,30.188161096092614,288.243647327029
16000,31.99999999999